In [ ]:
# !pip3 install Pillow --break-system-packages
# !pip3 install rembg --break-system-packages
# !pip3 install piexif --break-system-packages
# !pip3 install onnxruntime-gpu --break-system-packages

In [ ]:
# !pip3 install onnxruntime onnxruntime-gpu --break-system-packages

In [ ]:
import os
import piexif
import cv2
import itertools

import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm
from pprint import pprint
from pathlib import Path

from rembg import remove 
from PIL import Image 

# Functions

In [ ]:
# Get a list of all files and folders in the specified directory
def get_folders_in_directory(directory):
    items = os.listdir(directory)
    
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    
    return folders

def list_directory_contents(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    
    return items


In [ ]:
# Specify the directory path
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_100_30'

# Get the list of folders in the directory
folders = get_folders_in_directory(directory_path)
# folders = [x for x in folders if 'Subject' in x]
folders.sort()

# Print the list of folders
pprint(folders)

In [ ]:
subject_dic = {}

for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    
    # Get the list of files and folders in the directory
    contents = list_directory_contents(folder_path)
    contents = [x for x in contents if ':Zone.Identifier' not in x]
    
    subject_dic[folder] = contents
    
    for content in contents:
        folder_path_content = os.path.join(directory_path, folder, content)
        
        images = list_directory_contents(folder_path_content)
        images = [x for x in images if ':Zone.Identifier' not in x]
        
        print()
        print('#'*60)
        print(folder_path_content)
        print(images)
    

In [ ]:
subject_dic

# Using traditional models

In [ ]:
list_algorithm = [
    '',
    # '_colmap',
    # 'mve',
    # 'vggsfm'
    ]

list_scale_percent = [100, 50]
list_max_frames = [30, 45]

for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    print(f'Scale: {scale_percent}, Max Frames: {max_frames}')
    
    for subject in subject_dic.keys():    
        print('Subject', subject)
        folders = subject_dic[subject]
        
        for folder in folders:
            # When the input is raw
            # Convert to no bg (with/without resize)        
            path_input = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input'
            path_output = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input'

            path_input = os.path.join(f'{path_input}_{scale_percent}_{max_frames}', subject, folder)
            folder_path = Path(path_input)
                            
            ##################################################################
            # Loop through all files in the folder
            ##################################################################
            for file_path in tqdm(list(folder_path.iterdir()), total=len(list(folder_path.iterdir()))):
                if (
                    file_path.is_file() and 
                    ('.jpeg' in file_path.name) and 
                    (':Zone.Identifier' not in file_path.name)
                    ):

                    input_image_name = file_path.name
                    
                    file_path_input = os.path.join(path_input, input_image_name)
                    
                    # Processing the image 
                    # Removing the background from the given Image
                    input_img = Image.open(file_path_input)
                    input_img = remove(input_img) 
                    input_img = input_img.convert("RGB")

                    for algorithm in list_algorithm:
                        if algorithm == '':
                            file_path_output = os.path.join(f'{path_output}{algorithm}_{scale_percent}_{max_frames}_no_bg', subject, folder)  
                        else:
                            file_path_output = os.path.join(f'{path_output}{algorithm}_{scale_percent}_{max_frames}_no_bg', subject, folder, 'images')  
                    
                        if not os.path.exists(file_path_output):
                            os.makedirs(file_path_output)
                    
                        # print('')
                        # print(file_path_input)
                        # print(file_path_output)
                    
                        # Saving data
                        input_img.save(os.path.join(file_path_output, input_image_name))
                    
            